In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=RuntimeWarning)
from IPython.display import HTML, display # jupyter 사진넣기
import time                      # excution time 계산
from datetime import datetime   # system time 
import json                     # json save
# from imblearn.over_sampling import SMOTE
from sklearn.tree import export_graphviz  # 결정트리 모형
# import graphviz                         # 결정트리 모형
from sklearn import tree                  # 결정트리 모형  # Classifier tree
from sklearn.inspection import permutation_importance  # feature 중요도
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, Binarizer
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
                             confusion_matrix, precision_recall_curve, roc_curve)
from sklearn.model_selection import learning_curve, ShuffleSplit

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import (VotingClassifier, BaggingClassifier, RandomForestClassifier, AdaBoostClassifier,
                                ExtraTreesClassifier, GradientBoostingClassifier)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# import xgboost as xgb                # python wrapper
from xgboost import XGBClassifier    # sklearn wrapper
from lightgbm import LGBMClassifier  # sklearn wrapper

import pickle                         # 모델 저장
import matplotlib as mpl              # 한글깨짐
import matplotlib.font_manager as fm  # 한글깨짐
import matplotlib.font_manager        # 한글깨짐

# project

### 1단계 target 수(52만)만큼 train sample(52만) 합 약 104만건 + scaling 

In [5]:
df = pd.read_csv('data/NHIS_total_model.csv')
# # df = df.iloc[:10000, :]
# # print(df.info())
# # df.head(3)

In [6]:
df_copy = df.copy()
#     df_copy['target'] = df_copy['식전혈당(공복혈당)'].apply(lambda x: 1 if x >= 126 else ( 2 if 100 <= x < 126 else 3))
df_copy['target'] = df_copy['식전혈당(공복혈당)'].apply(lambda x: 1 if x >= 126 else 0 )
df_copy.drop('식전혈당(공복혈당)', axis=1, inplace=True)    
# 연령대 코드 --> 연령 decoding 후 drop
df_copy['연령'] =df_copy['연령대코드(5세단위)'].apply( lambda x: x*5 + 17 )
df_copy.drop('연령대코드(5세단위)', axis=1, inplace=True)
print(f"target   count : {df_copy[df_copy['target'] == 1].shape[0]} ({ round(df_copy[df_copy['target'] == 1].shape[0]/df_copy.shape[0], 4)*100 })%")
print(f"target외 count : {df_copy[df_copy['target'] != 1].shape[0]} ({ round(df_copy[df_copy['target'] != 1].shape[0]/df_copy.shape[0], 4)*100 })%")

target   count : 521858 (6.54)%
target외 count : 7453634 (93.46)%


In [7]:
# filter
target_count = df_copy[ df_copy['target'] == 1 ].shape[0]
not_target_count = df_copy[ df_copy['target'] != 1 ].shape[0]

# frac 설정 # if (target_count/not_target_count)*2 ==> target 33.3%비율(전체 156만)
frac=round( (target_count/not_target_count), 3) 
df_target = df_copy[ df_copy['target'] == 1 ]
df_X =  df_copy[ df_copy['target'] != 1 ].sample(frac=frac, random_state=1)

# concat
df = pd.concat ([df_target, df_X]).sort_values(by=['기준년도'])

target_count = df[df['target'] == 1].shape[0]
target_ex_count = df[df['target'] != 1].shape[0]
print(f"target   count : {target_count} ({ round( target_count/df.shape[0], 4)*100 })%")
print(f"target외 count : {target_ex_count} ({ round( target_ex_count/df.shape[0], 4)*100 })%")
# shuffle
import sklearn
df = sklearn.utils.shuffle(df)
# df=df.iloc[np.random.permutation(df.index)].reset_index(drop=True)
df.to_csv('./data/NHIS_model_1.csv', sep=',', encoding='utf-8', index=False)

target   count : 521858 (50.0)%
target외 count : 521754 (50.0)%


## 2단계 4:1 260만

In [9]:
df = pd.read_csv('data/NHIS_total_model.csv')
df_copy = df
df_copy['target'] = df_copy['식전혈당(공복혈당)'].apply(lambda x: 1 if x >= 126 else 0 )
df_copy.drop('식전혈당(공복혈당)', axis=1, inplace=True)    
# 연령대 코드 --> 연령 decoding 후 drop
df_copy['연령'] =df_copy['연령대코드(5세단위)'].apply( lambda x: x*5 + 17 )
df_copy.drop('연령대코드(5세단위)', axis=1, inplace=True)

# filter
target_count = df_copy[ df_copy['target'] == 1 ].shape[0]
not_target_count = df_copy[ df_copy['target'] != 1 ].shape[0]

# frac 설정 # if (target_count/not_target_count)*4 ==> target 33.3%비율(전체 156만)
frac=round( (target_count/not_target_count)*4, 3)  # 4:1 ==> total 260만
df_target = df_copy[ df_copy['target'] == 1 ]
df_X =  df_copy[ df_copy['target'] != 1 ].sample(frac=frac, random_state=1)

# concat
df = pd.concat ([df_target, df_X]).sort_values(by=['기준년도'])

target_count = df[df['target'] == 1].shape[0]
target_ex_count = df[df['target'] != 1].shape[0]
print(f"target   count : {target_count} ({ round( target_count/df.shape[0], 4)*100 })%")
print(f"target외 count : {target_ex_count} ({ round( target_ex_count/df.shape[0], 4)*100 })%")
# shuffle
import sklearn
df = sklearn.utils.shuffle(df)
# df=df.iloc[np.random.permutation(df.index)].reset_index(drop=True)
df.to_csv('./data/NHIS_model_2.csv', sep=',', encoding='utf-8', index=False)

target   count : 521858 (20.0)%
target외 count : 2087018 (80.0)%


##  3단계  3:1  208만

In [10]:
df = pd.read_csv('data/NHIS_total_model.csv')
df_copy = df
df_copy['target'] = df_copy['식전혈당(공복혈당)'].apply(lambda x: 1 if x >= 126 else 0 )
df_copy.drop('식전혈당(공복혈당)', axis=1, inplace=True)    
# 연령대 코드 --> 연령 decoding 후 drop
df_copy['연령'] =df_copy['연령대코드(5세단위)'].apply( lambda x: x*5 + 17 )
df_copy.drop('연령대코드(5세단위)', axis=1, inplace=True)

# filter
target_count = df_copy[ df_copy['target'] == 1 ].shape[0]
not_target_count = df_copy[ df_copy['target'] != 1 ].shape[0]

# frac 설정 # if (target_count/not_target_count)*2 ==> target 33.3%비율(전체 156만)
frac=round( (target_count/not_target_count)*3, 3)  # 3:1 ==> total 208만
df_target = df_copy[ df_copy['target'] == 1 ]
df_X =  df_copy[ df_copy['target'] != 1 ].sample(frac=frac, random_state=1)

# concat
df = pd.concat ([df_target, df_X]).sort_values(by=['기준년도'])

target_count = df[df['target'] == 1].shape[0]
target_ex_count = df[df['target'] != 1].shape[0]
print(f"target   count : {target_count} ({ round( target_count/df.shape[0], 4)*100 })%")
print(f"target외 count : {target_ex_count} ({ round( target_ex_count/df.shape[0], 4)*100 })%")
# shuffle
import sklearn
df = sklearn.utils.shuffle(df)
# df=df.iloc[np.random.permutation(df.index)].reset_index(drop=True)
df.to_csv('./data/NHIS_model_3.csv', sep=',', encoding='utf-8', index=False)

target   count : 521858 (25.0)%
target외 count : 1565263 (75.0)%


##  4단계  2:1  156만

In [11]:
df = pd.read_csv('data/NHIS_total_model.csv')
df_copy = df
df_copy['target'] = df_copy['식전혈당(공복혈당)'].apply(lambda x: 1 if x >= 126 else 0 )
df_copy.drop('식전혈당(공복혈당)', axis=1, inplace=True)    
# 연령대 코드 --> 연령 decoding 후 drop
df_copy['연령'] =df_copy['연령대코드(5세단위)'].apply( lambda x: x*5 + 17 )
df_copy.drop('연령대코드(5세단위)', axis=1, inplace=True)

# filter
target_count = df_copy[ df_copy['target'] == 1 ].shape[0]
not_target_count = df_copy[ df_copy['target'] != 1 ].shape[0]

# frac 설정 # if (target_count/not_target_count)*2 ==> target 33.3%비율(전체 156만)
frac=round( (target_count/not_target_count)*2, 3)  
df_target = df_copy[ df_copy['target'] == 1 ]
df_X =  df_copy[ df_copy['target'] != 1 ].sample(frac=frac, random_state=1)

# concat
df = pd.concat ([df_target, df_X]).sort_values(by=['기준년도'])

target_count = df[df['target'] == 1].shape[0]
target_ex_count = df[df['target'] != 1].shape[0]
print(f"target   count : {target_count} ({ round( target_count/df.shape[0], 4)*100 })%")
print(f"target외 count : {target_ex_count} ({ round( target_ex_count/df.shape[0], 4)*100 })%")
# shuffle
import sklearn
df = sklearn.utils.shuffle(df)
# df=df.iloc[np.random.permutation(df.index)].reset_index(drop=True)
df.to_csv('./data/NHIS_model_4.csv', sep=',', encoding='utf-8', index=False)

target   count : 521858 (33.339999999999996)%
target외 count : 1043509 (66.66)%


In [ ]:
## oneHotencoder